In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext
!pip install collections
!pip install tqdm
!pip install barbar
# !pip uninstall emoji
!pip install emoji==1.7
!pip install emojis
!pip install --upgrade urllib3
!pip install brokenaxes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227137 sha256=ad4c7e69e7b26fa07c923959ecbb77462c978d9c6e780e3d951d61a0f095bf27
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=51d9974bf3f68559fb893478ba7d01343e89bd984c5bfe05c5ad8fedcc6cfab8
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7ce

In [ ]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=e15016c4314f0a33f2afd22c8ff67fe3aaf15f3d334cc49269f55190197e0dd5
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


In [ ]:
!pip install transformers

In [ ]:
# !pip install pytorch_pretrained_bert

In [ ]:
!mkdir ckpts
!mkdir reports
!mkdir results

In [ ]:
import gensim
from tqdm import tqdm
from collections import defaultdict
import fasttext
import fasttext.util
# from gensim.models.utils_any2vec import ft_ngram_hashes  # This function is used to calculate hashes from ngrams to determine position in ngram matrix
from gensim.models.fasttext import ft_ngram_hashes

import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from brokenaxes import brokenaxes
from numpy.random import RandomState
import operator
from scipy.interpolate import make_interp_spline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import argparse
import numpy as np
import pandas as pd
import random
from transformers import AdamW, get_linear_schedule_with_warmup
from barbar import Bar
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
from emoji import UNICODE_EMOJI
import sys
import emojis
import numpy
import shutil

from statistics import mean
import math
from torch.autograd import Variable
%matplotlib inline
import gensim
import re
import numpy as np
from nltk import ngrams
from emoji import UNICODE_EMOJI
from nltk.tokenize import word_tokenize
from collections import defaultdict
from tqdm import tqdm_notebook
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
folder_path = 'Tuning'

In [ ]:
results_h = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h.jsonl'
results_h_2 = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_h_2.jsonl'
results_t = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_t.jsonl'
results_t_2 = f'drive/MyDrive/iSarcasm/{folder_path}/mix/results/results_t_2.jsonl'

In [ ]:
ls /content/drive/MyDrive/iSarcasm/fasttext

ft_250.model    ft_450.model    MultiTask/         results_h.jsonl
ft_250_swe.pth  ft_450_swe.pth  results_h_2.jsonl


In [ ]:
ft = gensim.models.KeyedVectors.load("/content/drive/MyDrive/iSarcasm/fasttext/ft_250.model")

In [ ]:
vector_size = 300

In [ ]:
class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()

        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss

In [ ]:
def get_vec_word(word1):
      flag = False
      try:
          idx = ft.key_to_index[word]
          vector2= ft.vectors[idx]
          return vector2
      except:
          tmp_v = np.random.uniform(-0.25, 0.25, 300)
          c = 0
          new_vector = np.zeros(vector_size, dtype=np.float32)
          char_ngrams = []
          word = '<' + word1 +'>'
          for n in range(5, 5 + 1):
              char_ngrams = [word[i:i + n] for i in range(len(word) - n + 1)]
              char_ngrams.append(word1)
              for x in char_ngrams:
                  try:
                    idx = ft.key_to_index[x]
                    vector2= ft.vectors[idx]
                    new_vector += vector2
                    c+=1
                  except:
                    pass
              all_zeros = not new_vector.any()
              if(all_zeros):
                new_vector = tmp_v
              else:
                new_vector/=c
              return new_vector

# split_data


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        self.xh = nn.Linear(input_size, hidden_size * 4, bias=bias)
        self.hh = nn.Linear(hidden_size, hidden_size * 4, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, input, hx=None):

        # Inputs:
        #       input: of shape (batch_size, input_size)
        #       hx: of shape (batch_size, hidden_size)
        # Outputs:
        #       hy: of shape (batch_size, hidden_size)
        #       cy: of shape (batch_size, hidden_size)

        if hx is None:
            hx = Variable(input.new_zeros(input.size(0), self.hidden_size))
            hx = (hx, hx)

        hx, cx = hx

        gates = self.xh(input) + self.hh(hx)

        # Get gates (i_t, f_t, g_t, o_t)
        input_gate, forget_gate, cell_gate, output_gate = gates.chunk(4, 1)

        i_t = torch.sigmoid(input_gate)
        f_t = torch.sigmoid(forget_gate)
        g_t = torch.tanh(cell_gate)
        o_t = torch.sigmoid(output_gate)

        cy = cx * f_t + i_t * g_t

        hy = o_t * torch.tanh(cy)


        return (hy, cy)


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bias, output_size):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = bias
        self.output_size = output_size

        self.rnn_cell_list = nn.ModuleList()

        self.rnn_cell_list.append(LSTMCell(self.input_size,
                                            self.hidden_size,
                                            self.bias))
        for l in range(1, self.num_layers):
            self.rnn_cell_list.append(LSTMCell(self.hidden_size,
                                                self.hidden_size,
                                                self.bias))

        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hx=None):

        # Input of shape (batch_size, seqence length , input_size)
        #
        # Output of shape (batch_size, output_size)

        if hx is None:
            if torch.cuda.is_available():
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size).cuda())
            else:
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size))
        else:
             h0 = hx

        outs = []

        hidden = list()
        for layer in range(self.num_layers):
            hidden.append((h0[layer, :, :], h0[layer, :, :]))

        for t in range(input.size(1)):

            for layer in range(self.num_layers):

                if layer == 0:
                    hidden_l = self.rnn_cell_list[layer](
                        input[:, t, :],
                        (hidden[layer][0],hidden[layer][1])
                        )
                else:
                    hidden_l = self.rnn_cell_list[layer](
                        hidden[layer - 1][0],
                        (hidden[layer][0], hidden[layer][1])
                        )

                hidden[layer] = hidden_l

            outs.append(hidden_l[0])

        out = outs[-1].squeeze()

        out = self.fc(out)

        return out

In [ ]:
def split_data(data_name):
      if(data_name == 'ArSarcasm'):
        df  = pd.read_csv('/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm/ArSarcasm_train.csv')
        folder = '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm/split/'
        df['sarcasm'] = df['sarcasm'].map({True: 1, False: 0})
        df.rename(columns = {'sarcasm':'sarcastic'}, inplace = True)

      elif(data_name == 'ArSarcasm_v2'):
        df  = pd.read_csv('/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/training_data.csv')
        folder = '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/'
        df['sarcasm'] = df['sarcasm'].map({True: 1, False: 0})
        df.rename(columns = {'sarcasm':'sarcastic'}, inplace = True)

      rng = RandomState()
      data = df.sample(frac=1).reset_index(drop=True)
      train, test = train_test_split(data, test_size=0.2, stratify=data["sarcastic"].values, random_state=42, shuffle=True)#

      train.to_csv('train.csv')
      test.to_csv('val.csv')

      shutil.copy('train.csv', f"{folder}train.csv")
      shutil.copy('val.csv', f"{folder}val.csv")


In [ ]:
def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [ ]:
def build_vocab(t_model):

  word2idx = {}
  # Add <pad> and <unk> tokens to the vocabulary
  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1

  d =t_model.vector_size
  # make it all zeroes and try
  # embeddings = np.random.uniform(-0.25, 0.25, (len(t_model.key_to_index)+2, d))
  embeddings = np.zeros( (len(t_model.key_to_index)+2, d))

  embeddings[word2idx['<pad>']] = np.zeros((d,))
  idx = 2
  for word in t_model.key_to_index:
      if word not in word2idx:
            word2idx[word] = idx
            embeddings[word2idx[word]] = np.array(get_vec_word(word), dtype=np.float32)
            idx += 1

  print(len(embeddings), len(word2idx))
  return  word2idx, embeddings

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df, static_input_ids2, pretraine_path='xlm-roberta-base', max_length=128):
        self.df = df
        self.max_length = max_length
        self.static_input_ids2= static_input_ids2
        self.tokenizer = AutoTokenizer.from_pretrained(pretraine_path)

    def __getitem__(self, index):
        text = self.df.iloc[index]['tweet']
        sarc_label = self.df.iloc[index]["sarcastic"]
        sent_label = self.df.iloc[index]["sentiment"]
        static_input_ids2 = self.static_input_ids2[index]
        encoded_input = self.tokenizer(
                text,
                max_length = self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors="pt",
            )

        input_ids = encoded_input["input_ids"]
        attention_mask = encoded_input["attention_mask"] if "attention_mask" in encoded_input else None

        data_input = {
            "input_ids":input_ids.flatten(),
            "attention_mask": attention_mask.flatten()
        }

        sarc_label_input ={
            "sarcasm": torch.tensor(sarc_label, dtype=torch.float),
        }
        sent_label_input ={
            "sentiment": torch.tensor(sent_label, dtype=torch.float),
        }
        static_input_ids ={
            "static_input_ids":torch.from_numpy(np.asarray(static_input_ids2)),
        }

        return data_input, sarc_label_input, sent_label_input, static_input_ids

    def __len__(self):
        return self.df.shape[0]

In [ ]:
def label_rule(mis_preds, cat_preds):
    for i in range(len(mis_preds)):
        if mis_preds[i] == 0:
            cat_preds[i] = 0
    return cat_preds


def accuracy(preds, y):
    all_output = preds.float().cpu()
    all_label = y.float().cpu()
    _, predict = torch.max(all_output, 1)
    acc = accuracy_score(all_label.numpy(), torch.squeeze(predict).float().numpy())
    return acc

def calc_accuracy(preds,y):
    predict = torch.argmax(preds, dim=1)
    accuracy = torch.sum(predict == y.squeeze()).float().item()
    return accuracy / float(preds.size()[0])

def calc_f1_sarcasm(preds,y):

    predict = torch.argmax(preds, dim=1)
    f1_sarcastic = f1_score(y, predict, average='binary', pos_label=1)
    return f1_sarcastic

def calc_f1_score(preds,y):
    predict = torch.argmax(preds, dim=1)
    f1_score_macro = f1_score(y, predict, average='macro')
    return f1_score_macro

def binary_accuracy2(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0))
    return acc

def binary_accuracy(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0) * y.size(1))
    return acc

In [ ]:
folder_dic = {
    "ArSarcasm": '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm',
    "ArSarcasm_v2": '/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2'
}

In [ ]:
folder_dic['ArSarcasm']

'/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm'

In [ ]:
dic= {
      "egypt": 'المصرية',
			"nile": 'المصرية',
			"msa": "اللغة العربية الفصحى",
			"magreb": "المغربية",
			"gulf": "الخليجية",
			"levant": "الشامية"
}


In [ ]:
def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

def preprocess(text):
    sent = add_space(text)
    sent = re.sub(r'(?:@[\w_]+)', "user", sent)
    sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
    sent = sent.replace('_', ' ')
    sent = sent.replace('#', ' ')
    return sent

def prepare_text(df, col):
    if col == 'tweet':
        df['dialect'] = df['dialect'].map(dic)
    for i in range(df.shape[0]):
        df.loc[i, col] = df.loc[i, 'dialect'] + ' [SEP] ' + df.loc[i, col]


    return df


In [ ]:
def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)




class TransformerLayer(nn.Module):
    def __init__(self,both=True,
                pretrained_path='aubmindlab/bert-base-arabert'):
        super(TransformerLayer, self).__init__()

        self.both = both
        self.transformer = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)


    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))

        return outputs

    def output_num(self):
        return self.transformer.config.hidden_size




In [ ]:
ls

ckpts/  drive/  reports/  results/  sample_data/


In [ ]:
def evaluate(base_model, mt_classifier, iterator, criterion):
    # initialize every epoch
    acc_sarcasm= 0
    f1_sarcasm =0
    f1_score_macro = 0
    loss= 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])


    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()
    with torch.no_grad():
        for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in sarc_label_input.items():
                sarc_label_input[k] = v.long().to(device)

            for k, v in sent_label_input.items():
                sent_label_input[k] = v.long().to(device)

            for k, v in static_input_ids.items():
                # print(k,v)
                static_input_ids[k] = v.to(device)


            sarcasm_target = sarc_label_input['sarcasm']

            # forward pass
            s_input_ids = static_input_ids['static_input_ids']
            output = base_model(**data_input)
            sarcasm_logits = mt_classifier(s_input_ids, output)
            # logits = sarcasm_logits[:,:2]

            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)

            # compute the loss
            loss_ = criterion(sarcasm_logits, sarcasm_target)

            # compute the running accuracy and losses
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            # loss_sarc += loss_sarcasm.item()

            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)


            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            #all_sarcasm_labels.extend(sarcasm_target.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())


    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)

    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)


    # accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro, 'report_sarcasm': report_sarcasm}
    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}
    losses = { 'loss': loss_ / len(iterator)}

    return accuracies, losses


In [ ]:
def predict(base_model, mt_classifier, iterator):
    # initialize every epoch
    acc_sarcasm= 0
    loss= 0

    f1_sarcasm =0
    f1_score_macro = 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()

    with torch.no_grad():
      for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in sarc_label_input.items():
                sarc_label_input[k] = v.long().to(device)

            for k, v in sent_label_input.items():
                sent_label_input[k] = v.long().to(device)
            for k, v in static_input_ids.items():
                static_input_ids[k] = v.to(device)



            # forward pass
            sarcasm_target = sarc_label_input['sarcasm']

            output = base_model(**data_input)
            s_input_ids = static_input_ids['static_input_ids']
            sarcasm_logits = mt_classifier(s_input_ids, output)

            # logits = sarcasm_logits[:,:2]
            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)

            # compute the loss
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)


            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())

    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)


    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)

    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}

    return accuracies, all_sarcasm_outputs, all_sarcasm_labels


In [ ]:
# 16_FL_1e-05_3_16826
batch_bert=16
loss_bert = 'FL'
lr_bert=1e-5
len_bert=64
epoch_bert=1
# [16826, 5338,
# 7930, 417,99]
seed_bert=16826
extra=f'_alpha_e_{epoch_bert}_{lr_bert}_{batch_bert}'

In [ ]:
dic_labels ={
    "NEG": 0,
    "NEU": 1,
    "POS": 2
}

In [ ]:
def loadTrainValData2(data_name="ArSarcasm_v2",size=0.2, batchsize=16, num_worker=0, pretraine_path="marbert", seed=42, max_length=128):

    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/train.csv"
    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')
    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]

    data = data[["tweet", "sarcastic", "sentiment"]]
    data = data.sample(frac=1).reset_index(drop=True)
    # data.head(10)
    df_train = data
    df_train = df_train.sample(frac=1).reset_index(drop=True)


    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/val.csv"


    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')

    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]
    # data.head(10)
    df_test = data
    # DF_train = TrainDataset(df_train, pretraine_path, max_length)
    # DF_test = TrainDataset(df_test, pretraine_path, max_length)

    # DF_train_loader = DataLoader(dataset=DF_train, batch_size=batchsize, shuffle=True,
    #                              num_workers=num_worker)
    # DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
    #                             num_workers=num_worker)
    # return DF_train_loader, DF_test_loader
    return df_train, df_test

# len(embeddings)

In [ ]:
def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.

    Args:
        texts (List[str]): List of text data

    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """
    tokenized_texts = []
    for sent in texts:
        tokenized_sent = word_tokenize(sent)
        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)
    return tokenized_texts

In [ ]:
def encode(tokenized_texts=None, word2idx=None, max_len=None):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []

    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))
        # Encode tokens to input_ids
        input_id = []
        # input_id = [(if token in word2idx: word2idx.get(token) else: input_id = word2idx.get('<unk>')) for token in tokenized_sent]
        for token in tokenized_sent:
            if token in word2idx:
              input_id_i = word2idx.get(token)
            else:
              input_id_i = word2idx.get('<unk>')
            input_id.append(input_id_i)

        input_ids.append(input_id)


    return input_ids

In [ ]:
class ATTClassifier(nn.Module):
    def __init__(self, in_feature, class_num_sarc=1, class_num_sent=3, dropout_prob=0.2):
        super(ATTClassifier, self).__init__()

        self.fc11 = nn.Linear(in_feature, 512)
        self.do1 = nn.Dropout(dropout_prob)
        self.r1= nn.ReLU()
        self.fc12  = nn.Linear(512, class_num_sarc)

        self.fc21 = nn.Linear(in_feature, 512)
        self.do2 = nn.Dropout(dropout_prob)
        self.r2= nn.ReLU()
        self.fc22= nn.Linear(512, class_num_sent)

        self.apply(init_weights)

    def forward(self, x):

        xx = x[1]
        out_sarc = self.fc12( self.r1 ( self.do1( self.fc11 (xx))))
        out_sent = self.fc22( self.r2 ( self.do2( self.fc21 (xx))))
        return [out_sarc,out_sent]

In [ ]:
def train(base_model, mt_classifier, iterator, optimizer,
          sarc_criterion, scheduler):

    # set the model in eval phase
    base_model.train(True)
    mt_classifier.train(True)

    acc_sarcasm= 0
    acc_sentiment= 0

    f1_sarcasm =0
    f1_score_macro = 0

    loss= 0

    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    all_sentiment_outputs = np.array([])
    all_sentiment_labels = np.array([])

    for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

        for k, v in data_input.items():
            # print("k",k)
            # print("v", v.shape)

            data_input[k] = v.to(device)
            # break

        for k, v in sarc_label_input.items():
            sarc_label_input[k] = v.long().to(device)

        for k, v in sent_label_input.items():
            sent_label_input[k] = v.long().to(device)

        for k, v in static_input_ids.items():
            # print("k",k)
            # print("v",len(v[0]))
            static_input_ids[k] = v.to(device)

        optimizer.zero_grad()


        #forward pass

        sarcasm_target = sarc_label_input['sarcasm']
        s_input_ids = static_input_ids['static_input_ids']


        # forward pass

        output = base_model(**data_input)

        sarcasm_logits = mt_classifier(s_input_ids, output )

        sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)


        # print(sarcasm_logits )
        # print(sarcasm_target)

        # print(sentiment_logits )
        # print(sentiment_target)
        # print(sentiment_probs)
        # print(torch.max(sentiment_probs, 1))

        loss_ = sarc_criterion(sarcasm_logits, sarcasm_target)
        # loss += loss_sarcasm.item()
        # loss += loss_sentiment.item()
        # backpropage the loss and compute the gradients
        loss_.backward()
        # loss_sarcasm.backward()
        # loss_sentiment.backward()

        optimizer.step()
        scheduler.step()

        acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
        _, predicted_sarcasm = torch.max(sarcasm_probs, 1)
        all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
        all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())




    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)



    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}
    losses = { 'loss': loss_ / len(iterator)}

    return accuracies, losses

In [ ]:
def train_full( config, train_loader, valid_loader):
    lr_o = config['lr_mult'] * config['lr']
    lr = config['lr']
    criterion = config['loss']
    # initialize models
    base_model = TransformerLayer(pretrained_path=config['pretrained_path'], both=True).to(device)
    classifier = ATTClassifier(base_model.output_num(), class_num_sarc=2, class_num_sent=3).to(device)

    # base_model.load_state_dict(torch.load(f"drive/MyDrive/iSarcasm/MultiTaskSWE/models/bert/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml64_{config['epochs']}_{config['args']['seed']}_sarcat.pth"))
    # classifier.load_state_dict(torch.load(f"drive/MyDrive/iSarcasm/MultiTaskSWE/models/bert/best_cls_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml64_{config['epochs']}_{config['args']['seed']}_sarcat.pth"))
    base_model.load_state_dict(torch.load(f"drive/MyDrive/iSarcasm/{folder_path}/bert/models/best_basemodel_sarcasm_{batch_bert}_{loss_bert}_{lr_bert}_ml{len_bert}_{epoch_bert}_{seed_bert}_sarcat_alpha.pth"))
    classifier.load_state_dict(torch.load(f"drive/MyDrive/iSarcasm/{folder_path}/bert/models/best_cls_sarcasm_{batch_bert}_{loss_bert}_{lr_bert}_ml{len_bert}_{epoch_bert}_{seed_bert}_sarcat_alpha.pth"))

    base_model = base_model.to(device)
    classifier = classifier.to(device)

    swe_marbert_model = SWE_MARBERT(base_model.output_num(),
                                    pretrained_embedding=embeddings,
                                    freeze_embedding=False,
                                    dropout=0.5)

    # Send model to `device` (GPU/CPU)
    swe_marbert_model = swe_marbert_model.to(device)

    swe_m_state_dict = swe_marbert_model.state_dict()

    # cnn_state_dict = {k: cnn_model.state_dict()[k] for k in filter(lambda x: x.startswith("embedding."), cnn_model.state_dict())}
    classifier_state_dict = {k: classifier.state_dict()[k] for k in filter(lambda x: x.startswith("fc11.") or x.startswith("fc21."), classifier.state_dict())}

    # swe_m_state_dict['embedding.weight'] = cnn_state_dict['embedding.weight']
    swe_m_state_dict['fc11.weight'] = classifier_state_dict['fc11.weight']
    swe_m_state_dict['fc21.weight'] = classifier_state_dict['fc21.weight']
    swe_m_state_dict['fc11.bias'] = classifier_state_dict['fc11.bias']
    swe_m_state_dict['fc21.bias'] = classifier_state_dict['fc21.bias']
    swe_marbert_model.load_state_dict(swe_m_state_dict, strict=False)

    # cls = 'ATTClassifier'

    # criterion = nn.CrossEntropyLoss().to(device)
    # criterion = FocalLoss_Ori(num_class=2, alpha=[1, 1], gamma=2).to(device)
    if criterion =='FL':
        criterion = FocalLoss_Ori(num_class=2, alpha=[0.6, 2.89], gamma=2).to(device)
    else:
        criterion = nn.CrossEntropyLoss().to(device)
    #
    #

    params = [{'params':base_model.parameters(), 'lr':config['lr']}, {'params': swe_marbert_model.parameters(), 'lr': config['lr']}]#, {'params':multi_task_loss.parameters(), 'lr': 0.0005}]
    optimizer = AdamW(params, lr=config["lr"])

    train_data_size = len(train_loader)
    steps_per_epoch = int(train_data_size / config['batch_size'])
    num_train_steps = len(train_loader) * config['epochs']
    warmup_steps = int(config['epochs'] * train_data_size * 0.1 / config['batch_size'])

    scheduler = get_linear_schedule_with_warmup(optimizer,  num_warmup_steps=warmup_steps, num_training_steps=num_train_steps)
    # Train model

    best_val_accuracy = 0
    best_val_f1_score_sarc = 0
    best_val_f1_sarcastic = 0

    best_val_accuracy_sent = 0
    best_val_f1_score_sent = 0

    best_val_metric = 0

    best_val_loss = float('+inf')

    best_report_sarcasm = None

    epo = 0


    for epoch in range(config['epochs']):
        # print("epoch {}".format(epoch + 1))

        train_accuracies, train_losses = train(base_model, swe_marbert_model, train_loader, optimizer, criterion,scheduler)
        valid_accuracies, valid_losses = evaluate(base_model, swe_marbert_model, valid_loader, criterion)

        val_loss = valid_losses['loss']

        total_val_metric = valid_accuracies['f1_score_sarc']
        print(epoch,valid_losses['loss'], valid_accuracies['acc_sarc'], valid_accuracies['f1_score_sarc'], valid_accuracies['f1_sarcastic'])
        if epoch == (config['epochs'] - 1):
        # if total_val_metric >= best_val_f1_score_sarc:

            epo = epoch+1
            best_val_loss = val_loss
            best_val_metric = total_val_metric

            best_val_f1_score_sarc = valid_accuracies['f1_score_sarc']
            best_val_f1_sarcastic = valid_accuracies['f1_sarcastic']

            # best_report_sarcasm = valid_accuracies['report_sarcasm']
            best_val_accuracy_sarc = valid_accuracies['acc_sarc']
            best_val_loss = valid_losses['loss']

            print("save model's checkpoint")
            torch.save(base_model.state_dict(), f"./ckpts/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth")
            torch.save(swe_marbert_model.state_dict(), f"./ckpts/best_swe_marbert_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth")


    return best_val_accuracy_sarc, best_val_f1_score_sarc, best_val_f1_sarcastic, best_val_loss,epo

In [ ]:
class SWE_MARBERT(nn.Module):
    def __init__(self,
                 in_feature,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 num_classes=2,
                 dropout=0.5,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100]):



        super(SWE_MARBERT, self).__init__()
        # Embedding layer

        self.vocab_size, self.embed_dim = pretrained_embedding.shape
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)


        self.fc11 = nn.Linear(in_feature, 512)
        self.fc21 = nn.Linear(in_feature, 512)

        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # self.lstm_model = LSTM(input_size=self.embed_dim, hidden_size=self.embed_dim, num_layers=1, bias=True , output_size=self.embed_dim)

        self.lstm_model = LSTM(input_size=np.sum(num_filters), hidden_size=np.sum(num_filters), num_layers=1, bias=True , output_size=np.sum(num_filters))

        # self.fc = nn.Linear( self.embed_dim + 2*512 , num_classes)
        self.fc = nn.Linear( np.sum(num_filters) + 2*512 , num_classes)



        self.dropout = nn.Dropout(p=dropout)
        # self.avg_pool = nn.AvgPool1d(3)

    def forward(self, input_ids, x):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """
        # print("input_ids", input_ids)
        # print("type input_ids", type(input_ids))
        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_reshaped = self.embedding(input_ids).float()
        #  Output shape: (b, embed_dim, max_len)
        x_reshaped = x_reshaped.permute(0, 2, 1)
        # #reshape it to(b, embed_dim)
        # x_reshaped = torch.mean(x_embed, dim=1)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]

        # # Concatenate x_pool_list to feed the fully connected layer.
        # # Output shape: (b, sum(num_filters))
        x_pool = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        x_pool  = x_pool.reshape([x_pool.shape[0], 1, x_pool.shape[1]])
        x_lstm = self.lstm_model(x_pool)

        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (b=8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))
        # print((x.keys()))
        xx = x[1]
        out_sarc = self.fc11 (xx)
        out_sent = self.fc21 (xx)
        x_fc = torch.cat([out_sarc, out_sent, x_lstm], dim=1)
        # x_fc = torch.cat([out_sarc, out_sent, x_pool], dim=1)

        # cls [b=8, 768]
        # x_fc = torch.cat([xx, xx, x_lstm], dim=1)
        # Compute logits. Output shape: (b, n_classes)
        # logits = self.fc(self.dropout(x_fc))
        logits = self.fc(self.dropout(x_fc))

        return logits


In [ ]:
def loadTestData(data_name, batchsize=16, num_worker=2, pretraine_path="xlm-roberta-base", max_length=128):

    path = "/content/drive/MyDrive/iSarcasm/Datasets3/ArSarcasm_v2/split/test.csv"

    data = pd.read_csv(path, encoding='utf-8')
    data['tweet'] = data['tweet'].apply(lambda x:preprocess(x))
    data = prepare_text(data, col='tweet')

    for i in range (len(data['sentiment'])):
        data['sentiment'][i] = dic_labels[data['sentiment'][i]]

    data_df = data[["tweet", "sarcastic", "sentiment"]]
    # DF_test = TrainDataset(data, pretraine_path, max_length)

    # DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
    #                             num_workers=num_worker)


    return data_df

In [ ]:
def predict(base_model, mt_classifier, iterator):
    # initialize every epoch
    acc_sarcasm= 0
    loss= 0

    f1_sarcasm =0
    f1_score_macro = 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()
    with torch.no_grad():
      for data_input, sarc_label_input, sent_label_input, static_input_ids  in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in sarc_label_input.items():
                sarc_label_input[k] = v.long().to(device)

            for k, v in sent_label_input.items():
                sent_label_input[k] = v.long().to(device)

            for k, v in static_input_ids.items():
                static_input_ids[k] = v.to(device)


            sarcasm_target = sarc_label_input['sarcasm']
            s_input_ids = static_input_ids['static_input_ids']
            # forward pass

            output = base_model(**data_input)

            sarcasm_logits = mt_classifier(s_input_ids, output )

            sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)



            # compute the loss
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)

            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())


    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro_sarc = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)



    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)

    accuracies = { 'acc_sarc': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score_sarc": fscore_macro_sarc}

    # accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro, 'report_sarcasm': report_sarcasm}
    return accuracies, all_sarcasm_outputs, all_sarcasm_labels

In [ ]:
def eval_full(config, loader1):
    criterion = config['loss']
    base_model = TransformerLayer(pretrained_path=config['pretrained_path'], both=True).to(device)
    # classifier = ATTClassifier(base_model.output_num(), class_num_sarc=2, class_num_sent=3).to(device)
    classifier = SWE_MARBERT(base_model.output_num(),
                                    pretrained_embedding=embeddings,
                                    freeze_embedding=False,
                                    dropout=0.5)

    base_model.load_state_dict(torch.load(f"./ckpts/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth"))
    classifier.load_state_dict(torch.load(f"./ckpts/best_swe_marbert_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth"))

    base_model = base_model.to(device)
    classifier = classifier.to(device)



    accuracies, all_outputs_pred, all_outputs_label = predict(base_model, classifier, loader1)
    return accuracies, all_outputs_pred, all_outputs_label


In [ ]:
def truncate(f, n):
    return math.floor(f * 10 ** n) / 10 ** n

In [ ]:
# 16_FL_1e-05_3_16826

In [ ]:
data_arr = [ "ArSarcasm_v2"]
lr_arr = [3e-5]
batch_arr = [32]
epoch_arr = [1]
loss_arr=['FL']
seed_arr=[16826]
data_i = data_arr[0]

In [ ]:
# 16_FL_1e-05_3_16826
batch_bert_arr=[32]
# loss_bert = ['FL']
lr_bert_arr=[3e-5]
epoch_bert_arr=[1,2]
# seed_bert=[16826]


In [ ]:
for lr_i in lr_arr:
  for batch_i in batch_arr:
    for epoch_i in epoch_arr:
      for loss_i in loss_arr:
        for seed_i in seed_arr:

          for lr_bert_i in lr_bert_arr:
            for batch_bert_i in batch_bert_arr:
              for epoch_bert_i in epoch_bert_arr:

                    batch_bert = batch_bert_i
                    lr_bert = lr_bert_i
                    epoch_bert = epoch_bert_i
                    extra=f'_alpha_e_{epoch_bert}_{lr_bert}_{batch_bert}'

                    args = {}
                    args['lr_mult'] = 1.0
                    args['seed'] = seed_i
                    args['num_worker'] = 4
                    args['lang'] = 'ar'
                    args['phase'] = 'train'
                    args['lm_pretrained'] = 'marbert'

                    args['lr'] =lr_i
                    args['epochs'] = epoch_i
                    args['batch_size'] = batch_i
                    args['loss'] = loss_i
                    config = {}
                    config["max_length"] = 70
                    max_length= config["max_length"]
                    config['args'] = args
                    config["output_for_test"] = True
                    config['epochs'] = args['epochs']
                    config["class_num"] = 1
                    config["lr"] = args['lr']
                    config['lr_mult'] = args['lr_mult']
                    config['batch_size'] = args['batch_size']
                    config['lm'] = args['lm_pretrained']
                    config['loss'] = args['loss']
                    lang = args['lang']
                    dosegmentation = False
                    if args['lm_pretrained'] == 'marbert':
                        config['pretrained_path'] = "UBC-NLP/MARBERT"

                    RANDOM_SEED = config['args']['seed']

                    random.seed(RANDOM_SEED)
                    np.random.seed(RANDOM_SEED)
                    torch.manual_seed(RANDOM_SEED)
                    torch.cuda.manual_seed(RANDOM_SEED)
                    torch.cuda.manual_seed_all(RANDOM_SEED)
                    torch.backends.cudnn.deterministic = True
                    torch.backends.cudnn.benchmark = False


                    model_name = f"{config['args']['batch_size']}_{config['args']['loss']}_{config['args']['lr']}_{config['args']['epochs']}_{config['args']['seed']}{extra}"
                    print('model_name', model_name)


                    df_train, df_val = loadTrainValData2(data_i, size = 0.2 ,batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'], max_length=config['max_length'])

                    train_texts = np.array(df_train['tweet'] )
                    val_texts = np.array(df_val['tweet'] )
                    train_labels = np.array(df_train['sarcastic'])
                    val_labels = np.array(df_val['sarcastic'])
                    word2idx, embeddings = build_vocab(ft)
                    embeddings = torch.tensor(embeddings)
                    tokenized_train_texts = tokenize(train_texts)
                    tokenized_val_texts = tokenize(val_texts)

                    train_input_ids = encode(tokenized_train_texts, word2idx, max_length)
                    val_input_ids = encode(tokenized_val_texts, word2idx, max_length)

                    # embeddings = load_pretrained_vectors(word2idx,t_model)
                    pretraine_path= config['pretrained_path']
                    DF_train = TrainDataset(df_train, train_input_ids, pretraine_path, max_length)
                    DF_val = TrainDataset(df_val, val_input_ids, pretraine_path, max_length)

                    batchsize = args['batch_size']

                    train_loader = DataLoader(dataset=DF_train, batch_size=batchsize, shuffle=True,
                                                    num_workers=0)
                    valid_loader = DataLoader(dataset=DF_val, batch_size=batchsize, shuffle=False,
                                                    num_workers=0)

                    best_val_accuracy_sarc, best_val_f1_score_sarc, best_val_f1_sarcastic, best_val_loss, epo  =train_full(config, train_loader, valid_loader)
                    # best_val_accuracy_sarc, best_val_f1_score_sarc, best_val_f1_sarcastic, epo

                    results = {'model_name' : f'{model_name}'}
                    results['loss_fn']= config['args']['loss']
                    results['batch_size']=config['args']['batch_size']
                    results['lr']=config['args']['lr']
                    results['f1_sarcastic'] = best_val_f1_sarcastic
                    results['f1_score_sarc'] = best_val_f1_score_sarc
                    results['epoch']=config['args']['epochs']
                    results['seed']=config['args']['seed']
                    results2 = {'model_name' : f'{model_name}'}
                    results2['loss_fn']= config['args']['loss']
                    results2['batch_size']=config['args']['batch_size']
                    results2['lr']=config['args']['lr']
                    results2['f1_sarcastic'] = truncate(best_val_f1_sarcastic,2)
                    results2['f1_score_sarc'] = truncate(best_val_f1_score_sarc,2)
                    results2['epoch']=config['args']['epochs']
                    results2['seed']=config['args']['seed']



                    args['phase'] = 'predict'
                    config['args'] = args

                    df_test = loadTestData(data_i,batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'],  max_length=config['max_length'])

                    test_texts = np.array(df_test['tweet'] )
                    test_labels = np.array(df_test['sarcastic'])
                    tokenized_test_texts = tokenize(test_texts)

                    test_input_ids = encode(tokenized_test_texts, word2idx, max_length)
                    DF_test = TrainDataset(df_test, test_input_ids, config['pretrained_path'], max_length)

                    test_loader = DataLoader(dataset=DF_test, batch_size=args['batch_size'], shuffle=False,
                                                    num_workers=0)
                    len(test_input_ids)
                    len(df_test)
                    DF_test

                    test_accuracies, test_all_outputs_sarc, test_all_labels_sarc = eval_full(config, loader1=test_loader)
                    results3 = {'model_name' : f'{model_name}'}
                    results3['loss_fn']= config['args']['loss']
                    results3['batch_size']=config['args']['batch_size']
                    results3['lr']=config['args']['lr']
                    results3['f1_sarcastic'] = test_accuracies['f1_sarcastic']
                    results3['f1_score_sarc'] = test_accuracies['f1_score_sarc']
                    results3['epoch']=config['args']['epochs']
                    results3['seed']=config['args']['seed']

                    results4 = {'model_name' : f'{model_name}'}
                    results4['loss_fn']= config['args']['loss']
                    results4['batch_size']=config['args']['batch_size']
                    results4['lr']=config['args']['lr']
                    results4['f1_sarcastic'] = truncate(test_accuracies['f1_sarcastic'],2)
                    results4['f1_score_sarc'] = truncate(test_accuracies['f1_score_sarc'],2)
                    results4['epoch']=config['args']['epochs']
                    results4['seed']=config['args']['seed']

                    with open(results_h, 'a+') as f:
                          f.write(json.dumps(results) + '\n')

                    with open(results_h_2, 'a+') as f:
                          f.write(json.dumps(results2) + '\n')

                    with open(results_t, 'a+') as f:
                          f.write(json.dumps(results3) + '\n')

                    with open(results_t_2, 'a+') as f:
                          f.write(json.dumps(results4) + '\n')

                    results_db_h = pd.read_json(results_h, lines=True)
                    print("results_db_h", results_db_h)
                    results_db_t = pd.read_json(results_t, lines=True)
                    print("results_db_t", results_db_t)
                    shutil.copy(f"./ckpts/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth", f"drive/MyDrive/iSarcasm/{folder_path}/mix/models/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth")
                    shutil.copy(f"./ckpts/best_swe_marbert_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth", f"drive/MyDrive/iSarcasm/{folder_path}/mix/models/best_swe_marbert_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat{extra}.pth")



model_name 32_FL_3e-05_1_16826_alpha_e_1_3e-05_32


Streaming output truncated to the last 5000 lines.
<ipython-input-32-873c24df68ce>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-32-873c24df68ce>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-32-873c24df68ce>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_lab

572931 572930


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

...

572931 572930


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


10038/10038: [===============================>] - ETA 0.9s
2510/2510: [===============================>] - ETA 0.1s
0 tensor(0.0015, device='cuda:0') 0.830131103074141 0.7555250536565552 0.6192619261926192
save model's checkpoint


Streaming output truncated to the last 5000 lines.
<ipython-input-39-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-39-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_labels[data['sentiment'][i]]
<ipython-input-39-446f52f76cbb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'][i] = dic_lab

3000/3000: [===============================>] - ETA 0.2s
results_db_h                                 model_name loss_fn  batch_size       lr  \
0   16_FL_1e-05_1_16826_alpha_e_1_1e-05_16      FL          16  0.00001   
1   16_FL_1e-05_1_16826_alpha_e_2_1e-05_16      FL          16  0.00001   
2   16_FL_3e-05_1_16826_alpha_e_1_3e-05_16      FL          16  0.00003   
3   16_FL_2e-05_1_16826_alpha_e_1_2e-05_16      FL          16  0.00002   
4   16_FL_3e-05_1_16826_alpha_e_2_3e-05_16      FL          16  0.00003   
5   16_FL_2e-05_1_16826_alpha_e_2_2e-05_16      FL          16  0.00002   
6   16_FL_4e-05_1_16826_alpha_e_1_4e-05_16      FL          16  0.00004   
7   16_FL_4e-05_1_16826_alpha_e_2_4e-05_16      FL          16  0.00004   
8   16_FL_5e-05_1_16826_alpha_e_1_5e-05_16      FL          16  0.00005   
9   32_FL_1e-05_1_16826_alpha_e_1_1e-05_32      FL          32  0.00001   
10  16_FL_5e-05_1_16826_alpha_e_2_5e-05_16      FL          16  0.00005   
11  32_FL_1e-05_1_16826_alpha_

In [ ]:
ls

ckpts/  drive/  reports/  results/  sample_data/


In [ ]:
results_db_h = pd.read_json(results_h, lines=True)
print("results_db_h", results_db_h)


results_db_h                                 model_name loss_fn  batch_size       lr  \
0   16_FL_1e-05_1_16826_alpha_e_1_1e-05_16      FL          16  0.00001   
1   16_FL_1e-05_1_16826_alpha_e_2_1e-05_16      FL          16  0.00001   
2   16_FL_3e-05_1_16826_alpha_e_1_3e-05_16      FL          16  0.00003   
3   16_FL_2e-05_1_16826_alpha_e_1_2e-05_16      FL          16  0.00002   
4   16_FL_3e-05_1_16826_alpha_e_2_3e-05_16      FL          16  0.00003   
5   16_FL_2e-05_1_16826_alpha_e_2_2e-05_16      FL          16  0.00002   
6   16_FL_4e-05_1_16826_alpha_e_1_4e-05_16      FL          16  0.00004   
7   16_FL_4e-05_1_16826_alpha_e_2_4e-05_16      FL          16  0.00004   
8   16_FL_5e-05_1_16826_alpha_e_1_5e-05_16      FL          16  0.00005   
9   32_FL_1e-05_1_16826_alpha_e_1_1e-05_32      FL          32  0.00001   
10  16_FL_5e-05_1_16826_alpha_e_2_5e-05_16      FL          16  0.00005   
11  32_FL_1e-05_1_16826_alpha_e_2_1e-05_32      FL          32  0.00001   
12  32_FL_2e

In [ ]:
results_db_t = pd.read_json(results_t, lines=True)
print("results_db_t", results_db_t)

results_db_t                                 model_name loss_fn  batch_size       lr  \
0   16_FL_1e-05_1_16826_alpha_e_1_1e-05_16      FL          16  0.00001   
1   16_FL_1e-05_1_16826_alpha_e_2_1e-05_16      FL          16  0.00001   
2   16_FL_1e-05_1_16826_alpha_e_3_1e-05_16      FL          16  0.00001   
3   16_FL_3e-05_1_16826_alpha_e_1_3e-05_16      FL          16  0.00003   
4   16_FL_2e-05_1_16826_alpha_e_1_2e-05_16      FL          16  0.00002   
5   16_FL_3e-05_1_16826_alpha_e_2_3e-05_16      FL          16  0.00003   
6   16_FL_2e-05_1_16826_alpha_e_2_2e-05_16      FL          16  0.00002   
7   16_FL_3e-05_1_16826_alpha_e_3_3e-05_16      FL          16  0.00003   
8   16_FL_2e-05_1_16826_alpha_e_3_2e-05_16      FL          16  0.00002   
9   16_FL_4e-05_1_16826_alpha_e_1_4e-05_16      FL          16  0.00004   
10  16_FL_4e-05_1_16826_alpha_e_2_4e-05_16      FL          16  0.00004   
11  16_FL_5e-05_1_16826_alpha_e_1_5e-05_16      FL          16  0.00005   
12  32_FL_1e